# VALIDATE REGIONCLIP ON COCO
Make sure to save checkpoints, embeddings and datasets to the correct file folders first.

In [ ]:
# RN50, GT, COCO
!python ./tools/train_net.py --eval-only --num-gpus 1 --config-file ./configs/COCO-InstanceSegmentation/CLIP_fast_rcnn_R_50_C4_ovd_zsinf.yaml "MODEL.WEIGHTS" "./pretrained_ckpt/regionclip/regionclip_pretrained-cc_rn50.pth" "MODEL.CLIP.TEXT_EMB_PATH" "./pretrained_ckpt/concept_emb/coco_65_cls_emb.pth" "MODEL.CLIP.CROP_REGION_TYPE" "GT" "MODEL.CLIP.MULTIPLY_RPN_SCORE" "False"  

In [ ]:
# RN50, RPN, COCO
!python ./tools/train_net.py --eval-only --num-gpus 1 --config-file ./configs/COCO-InstanceSegmentation/CLIP_fast_rcnn_R_50_C4_ovd_zsinf.yaml "MODEL.WEIGHTS" "./pretrained_ckpt/regionclip/regionclip_pretrained-cc_rn50.pth" "MODEL.CLIP.TEXT_EMB_PATH" "./pretrained_ckpt/concept_emb/coco_65_cls_emb.pth" "MODEL.CLIP.CROP_REGION_TYPE" "RPN" "MODEL.CLIP.MULTIPLY_RPN_SCORE" "True" "MODEL.CLIP.OFFLINE_RPN_CONFIG" "./configs/LVISv1-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml" "MODEL.CLIP.BB_RPN_WEIGHTS" "./pretrained_ckpt/rpn/rpn_lvis_866.pth"

# ZERO-SHOT OBJECT DETECTION WITH PRETRAINED REGIONCLIP ON CUSTOM DATASETS

Make sure to save checkpoints and embeddings to the correct file folders first.

## EXTRACT CONCEPT FEATURES
To create unnormalized text embeddings for class labels of custom dataset.

In [ ]:
# To extract features from ResNet50:
!python ./tools/extract_concept_features.py --config-file ./configs/LVISv1-InstanceSegmentation/CLIP_fast_rcnn_R_50_C4_zsinf.yaml "MODEL.WEIGHTS" "./pretrained_ckpt/regionclip/regionclip_pretrained-cc_rn50.pth" "MODEL.CLIP.OFFLINE_RPN_CONFIG" "./configs/LVISv1-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml" "INPUT_DIR" "./datasets/custom_concepts" "OUTPUT_DIR" "./output/concept_feats" "MODEL.CLIP.GET_CONCEPT_EMB" "True"                                                       

In [ ]:
# To extract features from ResNet50x4:
!python ./tools/extract_concept_features.py --config-file ./configs/LVISv1-InstanceSegmentation/CLIP_fast_rcnn_R_50_C4_zsinf.yaml "MODEL.WEIGHTS" "./pretrained_ckpt/regionclip/regionclip_pretrained-cc_rn50x4.pth" "MODEL.CLIP.TEXT_EMB_DIM" "640" "MODEL.RESNETS.DEPTH" "200" "MODEL.CLIP.OFFLINE_RPN_CONFIG" "./configs/LVISv1-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml" "INPUT_DIR" "./datasets/custom_concepts" "OUTPUT_DIR" "./output/concept_feats_rn50x4" "MODEL.CLIP.GET_CONCEPT_EMB" "True"

## (OPTIONALLY *EXTRACT REGION FEATURES) AND RUN INFERENCE

In [ ]:
# Inference
!python ./tools/train_net.py --eval-only --num-gpus 1 --config-file ./configs/COCO-InstanceSegmentation/CLIP_fast_rcnn_R_50_C4_ovd_zsinf.yaml "MODEL.WEIGHTS" "./pretrained_ckpt/regionclip/regionclip_pretrained-cc_rn50.pth" "MODEL.CLIP.TEXT_EMB_PATH" "./output/concept_feats/concept_embeds.pth" "MODEL.CLIP.CROP_REGION_TYPE" "RPN" "MODEL.CLIP.MULTIPLY_RPN_SCORE" "True" "MODEL.CLIP.OFFLINE_RPN_CONFIG" "./configs/LVISv1-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml" "MODEL.CLIP.BB_RPN_WEIGHTS" "./pretrained_ckpt/rpn/rpn_lvis_866.pth" "MODEL.ROI_HEADS.NUM_CLASSES" "17"

## To visualize:

Run the inference code again with "MODEL.CLIP.VIS" set to "True" then run the visualization code otherwise scores from 2 stages will be multiplied together to result in a confidence score > 100%.

In [ ]:
# Visualize results
!python ./tools/visualize_json_results.py --input ./output/inference/rpc_instances_results.json --output ./output/inference/rpc --dataset RPC_val --conf-threshold 0.05 --show-unique-boxes --max-boxes 25 --small-region-px 8100